## Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Data/weatherpy_database.csv")
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,Lebu,-37.6167,-73.6500,47.84,87,0,10.36,CL,clear sky
1,Deputatskiy,69.3000,139.9000,59.97,76,100,1.77,RU,overcast clouds
2,Mataura,-46.1927,168.8643,57.47,79,100,6.13,NZ,overcast clouds
3,Kruisfontein,-34.0033,24.7314,47.88,68,33,17.69,ZA,scattered clouds
4,Caravelas,-17.7125,-39.2481,69.57,89,7,5.41,BR,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
5,Rikitea,-23.1203,-134.9692,72.28,70,10,12.08,PF,clear sky
7,San Cristobal,7.7669,-72.2250,78.98,98,89,3.60,VE,overcast clouds
9,Albany,42.6001,-73.9662,84.92,78,44,5.30,US,scattered clouds
12,Ust-Nera,64.5667,143.2000,73.27,37,100,6.89,RU,overcast clouds
24,Vaitape,-16.5167,-151.7500,78.75,72,18,13.42,PF,few clouds
25,Butaritari,3.0707,172.7902,83.28,66,5,11.63,KI,clear sky
26,Fort Nelson,58.8053,-122.7002,72.99,31,75,14.97,CA,broken clouds
27,Naze,28.3667,129.4833,81.52,87,100,18.43,JP,overcast clouds
33,Umm Kaddadah,13.6017,26.6876,81.95,60,100,7.58,SD,overcast clouds
43,Hasaki,35.7333,140.8333,75.60,86,100,7.99,JP,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
5,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False
12,False,False,False,False,False,False,False,False,False
24,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
665,False,False,False,False,False,False,False,False,False
668,False,False,False,False,False,False,False,False,False
669,False,False,False,False,False,False,False,False,False
670,False,False,False,False,False,False,False,False,False


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Rikitea,PF,72.28,clear sky,-23.1203,-134.9692,
7,San Cristobal,VE,78.98,overcast clouds,7.7669,-72.2250,
9,Albany,US,84.92,scattered clouds,42.6001,-73.9662,
12,Ust-Nera,RU,73.27,overcast clouds,64.5667,143.2000,
24,Vaitape,PF,78.75,few clouds,-16.5167,-151.7500,
25,Butaritari,KI,83.28,clear sky,3.0707,172.7902,
26,Fort Nelson,CA,72.99,broken clouds,58.8053,-122.7002,
27,Naze,JP,81.52,overcast clouds,28.3667,129.4833,
33,Umm Kaddadah,SD,81.95,overcast clouds,13.6017,26.6876,
43,Hasaki,JP,75.60,overcast clouds,35.7333,140.8333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
     lat = row["Lat"]
     lng = row["Lng"]
    
     params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
     base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    
     hotels = requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
     try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
     except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Rikitea,PF,72.28,clear sky,-23.1203,-134.9692,People ThankYou
7,San Cristobal,VE,78.98,overcast clouds,7.7669,-72.2250,Posada Villaven C.A.
9,Albany,US,84.92,scattered clouds,42.6001,-73.9662,
12,Ust-Nera,RU,73.27,overcast clouds,64.5667,143.2000,Hotel Ust-Nera
24,Vaitape,PF,78.75,few clouds,-16.5167,-151.7500,Conrad Bora Bora Nui
25,Butaritari,KI,83.28,clear sky,3.0707,172.7902,Isles Sunset Lodge
26,Fort Nelson,CA,72.99,broken clouds,58.8053,-122.7002,The Fort Nelson Hotel
27,Naze,JP,81.52,overcast clouds,28.3667,129.4833,オーベルジュ・アガルイ
33,Umm Kaddadah,SD,81.95,overcast clouds,13.6017,26.6876,
43,Hasaki,JP,75.60,overcast clouds,35.7333,140.8333,Hotel Sunrise Choshi


In [9]:
hotel_df.dropna()
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Rikitea,PF,72.28,clear sky,-23.1203,-134.9692,People ThankYou
7,San Cristobal,VE,78.98,overcast clouds,7.7669,-72.2250,Posada Villaven C.A.
9,Albany,US,84.92,scattered clouds,42.6001,-73.9662,
12,Ust-Nera,RU,73.27,overcast clouds,64.5667,143.2000,Hotel Ust-Nera
24,Vaitape,PF,78.75,few clouds,-16.5167,-151.7500,Conrad Bora Bora Nui
...,...,...,...,...,...,...,...
665,Zavetnoye,RU,79.38,broken clouds,47.1194,43.8903,
668,Puerto Narino,CO,82.35,overcast clouds,-3.7703,-70.3831,Eware Refugio Amazonico
669,Oranjestad,AW,84.15,few clouds,12.5240,-70.0270,Divi Village Golf & Beach Resort
670,Maumere,ID,81.41,scattered clouds,-8.6199,122.2111,Sylvia Hotel


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace('', np.nan , inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Rikitea,PF,72.28,clear sky,-23.1203,-134.9692,People ThankYou
7,San Cristobal,VE,78.98,overcast clouds,7.7669,-72.2250,Posada Villaven C.A.
12,Ust-Nera,RU,73.27,overcast clouds,64.5667,143.2000,Hotel Ust-Nera
24,Vaitape,PF,78.75,few clouds,-16.5167,-151.7500,Conrad Bora Bora Nui
25,Butaritari,KI,83.28,clear sky,3.0707,172.7902,Isles Sunset Lodge
...,...,...,...,...,...,...,...
664,Jieshi,CN,82.76,overcast clouds,22.8134,115.8257,Yuehai Hotel
668,Puerto Narino,CO,82.35,overcast clouds,-3.7703,-70.3831,Eware Refugio Amazonico
669,Oranjestad,AW,84.15,few clouds,12.5240,-70.0270,Divi Village Golf & Beach Resort
670,Maumere,ID,81.41,scattered clouds,-8.6199,122.2111,Sylvia Hotel


In [11]:
# 8a. Create the output File (CSV)
vacation_data = "./WeatherPy_vacation.csv" 
# 8b. Export the City_Data into a csv
hotel_df.to_csv(vacation_data, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[['Lat', 'Lng']]

In [19]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[['Lat', 'Lng']]
max_temp = hotel_df['Max Temp']
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.75)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))